**A Crayon's Data Insights Knowledge Pill**
# Intro to Genetic Algorithms

We briefly introduce an implementation of a genetic algorithm by example and showcase how to solve sudokus using this approach.

This notebook and all accompanying code has been created for a knowledge exchange within Crayon's Data Insights Team.

### Table of Contents
1. [Introducing a Genetic Algorithm](#intro)
1. [Example Showcase: Solving Sudokus](#example)

<a id='intro'></a>
***
### 1. Introducing a Genetic Algorithm

Let's start with some intuition and fix terminology: Every `Organism` has a [chromosome](https://simple.wikipedia.org/wiki/Chromosome) and each `chromosome` is made up of `genes` [1]. Organisms can reproduce. Let's focus on organisms that need another `Organism` to reproduce and call it an `Individual`.

Without further ado let's create our first genetic algorithm from scratch. 

[1] The focus here is not biology and any usage of or relation between biological terms might be completely wrong. Mansoureh and Jill may be cringing right now. But - for the sake of this notebook - I live in ignorant bliss and don't care at all.

In [ ]:
%load_ext autoreload
%autoreload 2

import json

from genetic_algorithm_starter_kit.core import (
    Individual,
    Organism,
    Population,
)

Let's start by creating an organism.

In [ ]:
Organism.create()

You can also create an organism with a specific chromosome length.

In [ ]:
Organism.create(5)

There is a pool of possible "genes" that can be used for creating an organism in the background.

In [ ]:
Organism.GENES

It's boring if we are able to only create Organisms and can't do anything further. Let's build a metric to be able to tell whether a chromosome is good or bad, say, with respect to a reference chromosome. Let's judge it by how similar it is to a reference.

In [ ]:
def chromosome_similarity_score(chromosome: str, reference_chromosome: str) -> float:
    """Evaluate normalized similarity score for an individual w.r.t. a reference."""
    score = 0
    for idx, gene in enumerate(chromosome):
        if gene == reference_chromosome[idx]:
            score += 1
    return round(score/len(reference_chromosome), 5)

Let's check this newly minted metric in action.

In [ ]:
reference_chromosome = "hello, world!"
a = Organism.create(len(reference_chromosome))
score = chromosome_similarity_score(a.chromosome, reference_chromosome)

print(f"Reference:\t{reference_chromosome}\nOrganism:\t{a.chromosome}\nScore:\t\t{score} %")

Randomly created Organisms are mostly bad, i.e. score very low w.r.t. this new metric. How can we leverage the fact that some of the randomly created organisms score higher and use them to create new ones that score similarly well or even better? That's where genetic algorithms come into play!

The following cell introduces the class `Individual`. It too can create individuals of specific chromosome length.

In [ ]:
# Individuals can be created with specific chromosome length
chromosome_length = 50

ind0 = Individual.create(chromosome_length)
ind1 = Individual.create(chromosome_length)
ind0, ind1

The class `Individual` extends the functionality of `Organism` by being able to reproduce with a partner.

In [ ]:
child = ind0.reproduce(ind1)
child

With our metric defined earlier we can check the similarity - or fitness - of the child chromosome with its parent chromosomes.

In [ ]:
# Let's check how much was inherited from each of the two parent chromosomes
fitness0 = chromosome_similarity_score(child.chromosome, ind0.chromosome)
fitness1 = chromosome_similarity_score(child.chromosome, ind1.chromosome)

print(f"Parent 0:\t{fitness0}\nParent 1:\t{fitness1}\nMutation:\t{round(1-fitness0-fitness1, 5)}")

So the child chromosome is similar to its parent chromosomes. How would you implement `reproduce` such that genes can be inherited like in the case at hand? The implementation being used here is printed below.

```python
class Individual(Organism):

	prob_cutoffs = {
		"self": 0.45,
		"mate": 0.45 * 2,
	}

	def __init__(self, chromosome: str = "", *args):
		super().__init__(chromosome, *args)

	def reproduce(self, mate: Individual) -> Individual:
		"""Perform reproduction and produce new offspring."""
		child_chromosome = []
		for gene_self, gene_mate in zip(self.chromosome, mate.chromosome):
			prob = random.random()
			if prob < self.prob_cutoffs["self"]:
				# get gene from self
				child_chromosome.append(gene_self)
			elif prob < self.prob_cutoffs["mate"]:
				# get gene from mate
				child_chromosome.append(gene_mate)
			else:
				# random mutation
				child_chromosome.append(random.choice(super().GENES))
		return Individual("".join(child_chromosome))
```

In case of a reference - or *target* - chromosome we could simply create a LOT of individuals - a *population* - and see whether they are similar or not. The more complicated the target chromosome is the more unlikely it is to create it randomly (at least given only a limited amount of time and computational resources). Nevertheless, some will certainly be more similar - or better in this case - than others. Let's introduce some terminology and call the similarity here *fitness*, i.e. individuals with a more similar chromosome to the target are called fitter than others.

We can order all of the created individuals by their fitness. The next step is easy: Simply take the fittest individuals and let only them reproduce to get another set of individuals, i.e. the next generation. You can add further rules, e.g. take the fittest n-th percentile of a generation and copy them to the next generation et cetera, however, the important thing is to order the population by fitness and let only the fittest individuals reproduce offspring.

In [ ]:
# create a population and check out its individuals
pop = Population(size=10)
pop.individuals

In [ ]:
# Execute your first genetic algorithm
pop = Population(
    size=100,
    target_chromosome="hello, world!",
)
pop.evolve()

In [ ]:
# Load mystery chromosome
with open("etc/config.json") as f:
    config = json.load(f)

mystery_chromosome = config["target_chromosome"]

In [ ]:
# Execute genetic algorithm to breed for the mystery chromosome
pop = Population(
    size=500,
    target_chromosome=mystery_chromosome,
)
pop.evolve(verbose=False)

<a id='example'></a>
***
### 2. Example Showcase: Solving Sudokus

The previous example hints at the fact that a genetic algorithm can be used as an approach for optimization, however, it is also quite artificial as the target to be optimized for has to be provided. In this section we go over a more realistic and practical example by dropping this requirement and show how to use a genetic algorithm for solving a sudoku.

In [ ]:
from sudoku_solver.sudoku import Sudoku
from sudoku_solver.data.sudoku_configurations import valid_starting_position

Let's load a starting position.

In [ ]:
sudoku = Sudoku(valid_starting_position)
sudoku.board

To check for validity lateron we solve the sudoku (now with via backtracking algorithm) and save the solution into memory.

In [ ]:
sudoku.solve()
solved_board = sudoku.board
solved_board

In order to implement a genetic algorithm that solves a sudoku some metrics needs to be implemented - one for every rule that defines as solved sudoku. Check out the docstring of `Sudoku` below for a list of these rules.

In [ ]:
print(Sudoku.__doc__)

We can implement a fitness score for each of these rules which judges how many unique valid digits are i) in each row ii) in each column and
iii) in each box. Below are these scores printed for the starting position seen ealier. We can see that they are all equally bad.

In [ ]:
genetic_sudoku = Sudoku(valid_starting_position)
genetic_sudoku.fitness_score_boxes, genetic_sudoku.fitness_score_cols, genetic_sudoku.fitness_score_rows

So far there are 3 metrics and all are equally important, however, we need to choose a single metric [1] in order to judge which sudoku is better than another one so that a "fitter" subset of the population can be chosen for reproduction. Since all 3 rules must be satisfied in order for a sudoku to be classified as solved we cannot choose one of these metrics exclusively. Let's simply take their mean as an overall fitness score.


[1] Generally speaking, this is not a strict requirement but is more straightforward and builds upon the previously seen example building upon organisms, individuals, and populations.

In [ ]:
# Mean of the fitness scores for boxes, columns, and rows
genetic_sudoku.fitness_score

Finally let's see this genetic algorithm in action.

In [ ]:
# Solve sudoku using a genetic algorithm. Depending on the inputs this can take several minutes.
genetic_sudoku.evolve(
    population_size=5_000,
    lift_factor=0.25,
    verbose=False,
)

Finally, let's verify that the configuration found via the genetic algorithm is actually a solution.

In [ ]:
# Verify that the board is solved
genetic_sudoku.board == solved_board

Hooray, we're done! Hope you had fun.